<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [7]:
# use feature importance for feature selection
import numpy as np
import pandas as pd
from numpy import loadtxt
from numpy import sort
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

In [2]:
'''features = np.array(['last_auto_renew',
 'avg_songs_60days',
 'registered_via_7',
 'n_cancel',
 'avg_songs_90days',
 'total_songs',
 'total_mins',
 'avg_unq_60days',
 'avg_list_price',
 'registration_days',
 'avg_songs_compare60days',
 'n_trans',
 'avg_unq_90days',
 'avg_auto_renew',
 'avg_mins_60days',
 'n_trans_unq',
 'num_unq',
 'avg_secs_compare60days',
 'AVG_actual_paid',
 'avg_songs_compare90days',
 'avg_songs_compare30days',
 'avg_unq_compare60days',
 'avg_mins_90days',
 'avg_secs_compare30days',
 'avg_unq_compare30days',
 'avg_secs_compare90days',
 'avg_unq_compare90days',
 'avg_plan_days',
 'unlike_rate',
 'like_rate',
 'like_rate_60days',
 'city_median',
 'last_1_churn',
 'unlike_rate_60days',
 'unlike_rate_compare90days',
 'like_rate_compare30days',
 'unlike_rate_90days',
 'have_user_logs',
 'have_members',
 'last_2_churn',
 'bd_1',
 'last_3_churn',
 'last_4_churn',
 'last_5_churn',
 'bd_0',
 'unlike_rate_compare30days',
 'unlike_rate_compare60days',
 'like_rate_compare60days',
 'like_rate_compare90days',
 'city_big',
 'city_other',
 'city_small',
 'registered_via_0',
 'registered_via_9',
 'registered_via_other',
 'bd_-1',
 'like_rate_90days'])'''

"features = np.array(['last_auto_renew',\n 'avg_songs_60days',\n 'registered_via_7',\n 'n_cancel',\n 'avg_songs_90days',\n 'total_songs',\n 'total_mins',\n 'avg_unq_60days',\n 'avg_list_price',\n 'registration_days',\n 'avg_songs_compare60days',\n 'n_trans',\n 'avg_unq_90days',\n 'avg_auto_renew',\n 'avg_mins_60days',\n 'n_trans_unq',\n 'num_unq',\n 'avg_secs_compare60days',\n 'AVG_actual_paid',\n 'avg_songs_compare90days',\n 'avg_songs_compare30days',\n 'avg_unq_compare60days',\n 'avg_mins_90days',\n 'avg_secs_compare30days',\n 'avg_unq_compare30days',\n 'avg_secs_compare90days',\n 'avg_unq_compare90days',\n 'avg_plan_days',\n 'unlike_rate',\n 'like_rate',\n 'like_rate_60days',\n 'city_median',\n 'last_1_churn',\n 'unlike_rate_60days',\n 'unlike_rate_compare90days',\n 'like_rate_compare30days',\n 'unlike_rate_90days',\n 'have_user_logs',\n 'have_members',\n 'last_2_churn',\n 'bd_1',\n 'last_3_churn',\n 'last_4_churn',\n 'last_5_churn',\n 'bd_0',\n 'unlike_rate_compare30days',\n 'unlik

In [3]:
'''importance = np.array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.0006699 , 0.00098303, 0.00121894, 0.00242734, 0.00342978,
       0.00363051, 0.00410209, 0.00417466, 0.00529354, 0.00680448,
       0.00685531, 0.00700479, 0.00706738, 0.00726252, 0.00784994,
       0.00814589, 0.00857561, 0.00888073, 0.01093964, 0.01479251,
       0.01760849, 0.01824646, 0.01931317, 0.019883  , 0.02022266,
       0.02126939, 0.02554749, 0.02957831, 0.0333267 , 0.03658906,
       0.03940495, 0.04246793, 0.05384875, 0.05773023, 0.08601544,
       0.09623662, 0.2626028 ])'''

'importance = np.array([0.        , 0.        , 0.        , 0.        , 0.        ,\n       0.        , 0.        , 0.        , 0.        , 0.        ,\n       0.        , 0.        , 0.        , 0.        , 0.        ,\n       0.        , 0.        , 0.        , 0.        , 0.        ,\n       0.0006699 , 0.00098303, 0.00121894, 0.00242734, 0.00342978,\n       0.00363051, 0.00410209, 0.00417466, 0.00529354, 0.00680448,\n       0.00685531, 0.00700479, 0.00706738, 0.00726252, 0.00784994,\n       0.00814589, 0.00857561, 0.00888073, 0.01093964, 0.01479251,\n       0.01760849, 0.01824646, 0.01931317, 0.019883  , 0.02022266,\n       0.02126939, 0.02554749, 0.02957831, 0.0333267 , 0.03658906,\n       0.03940495, 0.04246793, 0.05384875, 0.05773023, 0.08601544,\n       0.09623662, 0.2626028 ])'

In [4]:
#importance_sort = [importance[i] for i in np.argsort(-importance)]

In [9]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
X_train = train.drop(columns=['is_churn']).values
y_train=train.is_churn.values
X_test = test.drop(columns=['is_churn']).values
y_test=test.is_churn.values

In [ ]:
# fit model on all training data
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)

In [ ]:
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = XGBClassifier()
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))